In [1]:
try:
    from variables import Variable
except:
    from AutoDiff.variables import Variable
try:
    from vectorize_func import vectorize_variable
except:
    from AutoDiff.vectorize_func import vectorize_variable
try:
    import AD_numpy as anp
except:
    import AutoDiff.AD_numpy as anp
import numpy as np
from numpy.linalg import pinv
from numpy.linalg import norm

### Newton Method for Scalar Function

In [2]:
def newton_method_scalar(fn, initial_val, threshold, max_iter, verbose=True):
    
    # create initial variables
    # right now we only test with the 26 alphabets
    from string import ascii_lowercase
    import pandas as pd
    
    name_ls = iter(ascii_lowercase)
    
    # create initial variables
    var_names = []
    var = []
    for i in initial_val:
        name = next(name_ls)
        var.append(Variable(name, i))
        var_names.append(name)
    
    val = np.array(initial_val)
    nums_iteration = 1
    while True:
        val_new = val - fn(*val) / list(fn(*var).der.values())
        # recreate new variables with new values
        var = []
        for i, v in enumerate(val_new):
            var.append(Variable(var_names[i], v))
            
        # print iteration output
        if verbose is True:
            print(f'Iteration at {nums_iteration}, at {val_new} ')
        
        # threshold stopping condition 
        if np.sqrt(np.sum((val_new - val)**2)) < threshold:
            print(f'After {nums_iteration} iterations, found a root: {val_new}')
            break
        
        # iteration stopping condition
        if nums_iteration >= max_iter:
            break
        nums_iteration +=1
        val = val_new

In [3]:
# an example
f = lambda x, y, z: (x-4)**2 + (y-3)**2 + (z-2)**2
newton_method_scalar(f, [3, 2, 1], 1e-6, 50, verbose=True)

Iteration at 1, at [4.5 3.5 2.5] 
Iteration at 2, at [3.75 2.75 1.75] 
Iteration at 3, at [4.125 3.125 2.125] 
Iteration at 4, at [3.9375 2.9375 1.9375] 
Iteration at 5, at [4.03125 3.03125 2.03125] 
Iteration at 6, at [3.984375 2.984375 1.984375] 
Iteration at 7, at [4.0078125 3.0078125 2.0078125] 
Iteration at 8, at [3.99609375 2.99609375 1.99609375] 
Iteration at 9, at [4.00195312 3.00195312 2.00195312] 
Iteration at 10, at [3.99902344 2.99902344 1.99902344] 
Iteration at 11, at [4.00048828 3.00048828 2.00048828] 
Iteration at 12, at [3.99975586 2.99975586 1.99975586] 
Iteration at 13, at [4.00012207 3.00012207 2.00012207] 
Iteration at 14, at [3.99993896 2.99993896 1.99993896] 
Iteration at 15, at [4.00003052 3.00003052 2.00003052] 
Iteration at 16, at [3.99998474 2.99998474 1.99998474] 
Iteration at 17, at [4.00000763 3.00000763 2.00000763] 
Iteration at 18, at [3.99999619 2.99999619 1.99999619] 
Iteration at 19, at [4.00000191 3.00000191 2.00000191] 
Iteration at 20, at [3.999999

### Newton Method for Vector Function

In [4]:
def newton_method_vector(fn, initial_val, threshold, max_iter, verbose=True):
    
    # create initial variables
    # right now we only test with the 26 alphabets
    from string import ascii_lowercase
    
    name_ls = iter(ascii_lowercase)
    
    # create initial variables
    var_names = []
    var = []
    for i in initial_val:
        name = next(name_ls)
        var.append(Variable(name, i))
        var_names.append(name)

    val = np.array(initial_val)
    nums_iteration = 1
    while True:  
        val_vector = fn(*var).val
        jacobian = fn(*var).jacobian().values
        d_x = np.dot(pinv(jacobian), val_vector)
        
        # update Variables after each iteration
        var = []
        for i in range(len(initial_val)):
            val[i] = val[i] - d_x[i]
            var.append(Variable(var_names[i], val[i]))
        
        # print iteration output
        if verbose is True:
            print(f'Iteration at {nums_iteration}, at {val}')
        
        # threshold stopping condition 
        if norm(fn(*var).val) < threshold:
            print(f'After {nums_iteration} iterations, found a root: {val}')
            break
        
        # iteration stopping condition
        if nums_iteration >= max_iter:
            break
        nums_iteration +=1
    return val

In [5]:
# A user can freely modify the number of functions and the number of inputs. 
# For example, if one wants to have 2 functions with 3 inputs, a code below is an example. 

@vectorize_variable
def vec_fn(x, y, z):
    f1 = anp.cos(x) + anp.sin(y)+ anp.cos(z) 
    f2 = x**2 - y**2 - z**2
    return np.array([f1,f2])
x = Variable('x', np.pi)
y = Variable('y', np.pi)
z = Variable('z', np.pi)
f = vec_fn(x,y,z)

In [6]:
# Now, user can find a root.
root = newton_method_vector(vec_fn, [np.pi, np.pi, np.pi], 1e-6, 50, verbose=True)

[-2.        -9.8696044]
Iteration at 1, at [2.92699082 1.14159265 3.35619449]
[-1.0448251 -4.       ]
Iteration at 2, at [3.63230984 4.60654298 2.19681605]
[ -2.46232257 -12.85256422]
Iteration at 3, at [ 4.61038753  5.19725699 -0.34993808]
[-0.0471672  -5.87826369]
Iteration at 4, at [ 4.83192692  4.82804193 -0.35324188]
[ 0.06418995 -0.08725097]
Iteration at 5, at [ 4.7844425   4.76931199 -0.38292099]
[ 0.00118809 -0.00207529]
Iteration at 6, at [ 4.78355737  4.76815504 -0.38356182]
[ 4.49664438e-07 -9.65764036e-07]
Iteration at 7, at [ 4.78355704  4.76815458 -0.38356207]
After 7 iterations, found a root: [ 4.78355704  4.76815458 -0.38356207]


In [7]:
# check if answers are correct
np.cos(root[0]) + np.sin(root[1]) + np.cos(root[2])

7.116529587847253e-14

In [8]:
# check if answers are correct
root[0]**2 - root[1]**2 - root[2]**2

-1.5457080060343742e-13